In [26]:
import numpy as np
import pandas as pd

In [27]:
train = pd.read_csv('train_users_2.csv')

In [28]:
test = pd.read_csv('test_users.csv')

In [29]:
train.shape

(213451, 16)

In [30]:
test.shape

(62096, 15)

In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
id                         213451 non-null object
date_account_created       213451 non-null object
timestamp_first_active     213451 non-null int64
date_first_booking         88908 non-null object
gender                     213451 non-null object
age                        125461 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
dtypes: float64(1), int64(2), object(13)
memory usage: 26.1+ MB


In [32]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62096 entries, 0 to 62095
Data columns (total 15 columns):
id                         62096 non-null object
date_account_created       62096 non-null object
timestamp_first_active     62096 non-null int64
date_first_booking         0 non-null float64
gender                     62096 non-null object
age                        33220 non-null float64
signup_method              62096 non-null object
signup_flow                62096 non-null int64
language                   62096 non-null object
affiliate_channel          62096 non-null object
affiliate_provider         62096 non-null object
first_affiliate_tracked    62076 non-null object
signup_app                 62096 non-null object
first_device_type          62096 non-null object
first_browser              62096 non-null object
dtypes: float64(2), int64(2), object(11)
memory usage: 7.1+ MB


In [33]:
train_row = train.shape[0]

In [34]:
train_row

213451

In [35]:
labels = np.unique(train['country_destination'].values)

In [36]:
labels

array(['AU', 'CA', 'DE', 'ES', 'FR', 'GB', 'IT', 'NDF', 'NL', 'PT', 'US',
       'other'], dtype=object)

In [37]:
target = train['country_destination']

In [39]:
train.drop(['country_destination','date_first_booking'],axis = 1,inplace=True)
test.drop(['date_first_booking'],axis=1,inplace=True)

In [41]:
train.shape

(213451, 14)

In [42]:
test.shape

(62096, 14)

In [43]:
df = pd.concat([train,test],axis=0,ignore_index = True)

In [44]:
df.shape

(275547, 14)

In [50]:
df_sub = df.loc[:999,:]

In [51]:
df_sub.shape

(1000, 14)

In [54]:
df_sub.columns

Index(['id', 'date_account_created', 'timestamp_first_active', 'gender', 'age',
       'signup_method', 'signup_flow', 'language', 'affiliate_channel',
       'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
       'first_device_type', 'first_browser'],
      dtype='object')

In [55]:
dac = df_sub.date_account_created

In [58]:
tfa = df_sub.timestamp_first_active

In [139]:
age = df_sub.age

# date_account_created处理

In [65]:
dac = pd.to_datetime(dac)

In [73]:
#提取特征year\month\day
dac_year = np.array([x.year for x in dac])
dac_month = np.array([x.month for x in dac])
dac_day = np.array([x.day for x in dac])

In [182]:
#提取特征week
dac_week = np.array([x.isoweekday() for x in dac])
dac_week_dummy = pd.get_dummies(dac_week,prefix='dac_week')

In [87]:
from datetime import date
Y=2000
seasons = [(0,(date(Y,1,1),date(Y,3,20))),#'winter'
           (1,(date(Y,3,21),date(Y,6,20))),#'spring'
           (2,(date(Y,6,21),date(Y,9,22))),#'summer'
           (3,(date(Y,9,23),date(Y,12,20))),#'autumn'
           (0,(date(Y,12,21),date(Y,12,31)))]#'winter'
def get_season(dt):
    dt = dt.date()
    dt = dt.replace(year=Y)
    return next(season for season,(start,end) in seasons if start<=dt<=end)

In [90]:
#提取特征季节，并进行one-hot处理
dac_season = np.array([get_season(x) for x in dac])
dac_season_dummy = pd.get_dummies(dac_season,prefix='dac_season')

# timestamp_first_active处理

In [98]:
tfa[0]

20090319043255

In [100]:
#首先进行日期的分割
import datetime
tfa = tfa.astype(str).apply(lambda x:datetime.datetime(
    int(x[:4]),int(x[4:6]),int(x[6:8]),int(x[8:10]),int(x[10:12]),int(x[12:])
                                                      ))

In [102]:
#year\month\day
tfa_year = np.array([x.year for x in tfa])
tfa_month = np.array([x.month for x in tfa])
tfa_day = np.array([x.day for x in tfa])

In [106]:
#weekday+onehot
tfa_week = np.array([x.isoweekday() for x in tfa])
tfa_week_dummy = pd.get_dummies(tfa_week,prefix='tfa_week')

In [110]:
#season+onehot
tfa_season = np.array([get_season(x) for x in tfa])
tfa_season_dummy = pd.get_dummies(tfa_season,prefix='tfa_season')

# date_account_created-timestamp_first_active

In [169]:
dt_span = np.array([(x-y).days for x,y in zip(dac,tfa)])

In [171]:
np.unique(dt_span)

array([  -1,    5,    6,  100,  162,  215,  261,  279,  307,  311,  355,
        465,  475,  664,  731,  764, 1368, 1455])

In [172]:
#将结果划分为一天(-1) 、 一月(-1~30)、 一年(30~265)和其他
def get_span(dt):
    if dt == -1:
        return 'OneDay'
    elif dt<30 and dt>-1:
        return 'OneMonth'
    elif dt<=365 and dt>=30:
        return 'OneYear'
    else:
        return 'other'

In [173]:
dt_span = np.array([ get_span(x) for x in dt_span])

In [175]:
dt_span_dummy = pd.get_dummies(dt_span,prefix='dt_span')

# age

In [140]:
#将误写为年份的数据进行预处理
av = age.values

In [141]:
av = np.where(np.logical_and(av<2000,av>1900),2014-av,av)

D:\Users\dr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.
D:\Users\dr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


In [142]:
#空值年龄填充为-1
age.fillna(-1,inplace=True)

D:\Users\dr\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [144]:
#对年龄进行分段，将连续型转换为离散型
div = 15
def get_age(age):
    if age<0:
        return  'NA'
    elif age<div:
        return div
    elif age<=div*2:
        return div*2
    elif age<=div*3:
        return div*3
    elif age<=div*4:
        return div*4
    elif age<=div*5:
        return div*5
    elif age<=110:
        return div*6
    else:
        return 'Unphysical'

In [145]:
#age分段+one-hot
age_gap = np.array([get_age(x) for x in age])
age_gap_dummy = pd.get_dummies(age_gap,prefix='age_gap')

# 将以上的处理好的数据进行删除和合并

In [185]:
df_new = df_sub

In [190]:
drop_lolumns = ['date_account_created','timestamp_first_active','age']

In [191]:
for item in drop_lolumns:
    df_new.drop([item],axis=1,inplace=True)

D:\Users\dr\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [195]:
insert_columns = [dac_week_dummy,dac_season_dummy,
                  tfa_week_dummy,tfa_season_dummy,
                  dt_span_dummy,
                  age_gap_dummy
                 ]

In [196]:
for item in insert_columns:
    df_new = pd.concat((df_new,item),axis=1)

In [197]:
df_new.shape

(1000, 45)

# 原数据中其他的特征，统一进行one-hot处理

In [184]:
feat_other = ['gender',
              'signup_method',
              'signup_flow',
              'language',
              'affiliate_channel', 
              'affiliate_provider',
              'first_affiliate_tracked', 
              'signup_app',
              'first_device_type',
              'first_browser'
             ]

In [199]:
for f in feat_other:
    f_dummy = pd.get_dummies(df_new[f],prefix=f,dummy_na=True)
    df_new.drop([f],axis=1,inplace=True)
    df_new = pd.concat((df_new,f_dummy),axis=1)

In [200]:
df_new.shape

(1000, 98)